In [1]:
import torch

from sample_factory.cfg.arguments import load_from_checkpoint
from sample_factory.utils.attr_dict import AttrDict
from sample_factory.algo.utils.env_info import extract_env_info
from sample_factory.algo.utils.make_env import make_env_func_batched
from sample_factory.model.model_utils import get_rnn_size

from sf_examples.nethack.train_nethack import parse_nethack_args, register_nethack_components, make_nethack_actor_critic

/home/bartek/Workspace/ideas/sample-factory/sf_examples/nethack/render_utils/Hack-Regular.ttf


In [2]:
env_name = "challenge"
register_nethack_components()
cfg = parse_nethack_args(
    [
        f"--env={env_name}",
        "--model=ScaledNet",
        "--use_resnet=True",
        "--h_dim=1738",
        "--rnn_size=1738",
        "--use_pretrained_checkpoint=True",
        "--model_path=/home/bartek/Workspace/data/sf_checkpoints/amzn-AA-BC/pretrained_use_prev_action",
        "--critic_add_layernorm=True",
        "--use_prev_action=True",
        "--actor_critic_share_weights=False",
    ]
)

[2024-05-23 12:14:15,550][13622] register_encoder_factory: <function make_nethack_encoder at 0x796bcf8df1c0>
[2024-05-23 12:14:15,551][13622] register_actor_critic_factory: <function make_nethack_actor_critic at 0x796bcf8df370>


In [3]:
env = make_env_func_batched(cfg, env_config=AttrDict(worker_index=0, vector_index=0, env_id=0))
env_info = extract_env_info(env, cfg)

obs_space = env_info.obs_space
action_space = env.action_space
obs, info = env.reset(seed=0)


In [5]:
obs["message"]

tensor([[ 89, 111, 117,  32,  97, 114, 101,  32, 108, 117,  99, 107, 121,  33,
          32,  32,  70, 117, 108, 108,  32, 109, 111, 111, 110,  32, 116, 111,
         110, 105, 103, 104, 116,  46,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0

In [6]:
shared_layer_rnn_states = torch.zeros([1, get_rnn_size(cfg)], dtype=torch.float32)
shared_layer_model = make_nethack_actor_critic(cfg, obs_space, action_space)

[2024-05-23 12:14:19,279][13622] RunningMeanStd input shape: (1,)
[2024-05-23 12:14:20,001][13622] RunningMeanStd input shape: (1,)
[2024-05-23 12:14:20,325][13622] Loading state from checkpoint /home/bartek/Workspace/data/sf_checkpoints/amzn-AA-BC/pretrained_use_prev_action/checkpoint_p0/checkpoint_000000000_0.pth...
[2024-05-23 12:14:20,418][13622] RunningMeanStd input shape: (1,)
[2024-05-23 12:14:24,685][13622] Loading model from pretrained checkpoint


In [7]:
cfg = parse_nethack_args(
    [
        f"--env={env_name}",
        "--model=ScaledNet",
        "--use_resnet=True",
        "--h_dim=1738",
        "--rnn_size=1738",
        "--use_pretrained_checkpoint=True",
        "--model_path=/home/bartek/Workspace/data/sf_checkpoints/amzn-AA-BC/pretrained_use_prev_action",
        "--use_prev_action=True",
    ]
)

In [8]:
rnn_states = torch.zeros([1, get_rnn_size(cfg)], dtype=torch.float32)
model = make_nethack_actor_critic(cfg, obs_space, action_space)

[2024-05-23 12:14:24,712][13622] RunningMeanStd input shape: (1,)
[2024-05-23 12:14:25,012][13622] Loading state from checkpoint /home/bartek/Workspace/data/sf_checkpoints/amzn-AA-BC/pretrained_use_prev_action/checkpoint_p0/checkpoint_000000000_0.pth...
[2024-05-23 12:14:25,101][13622] Loading model from pretrained checkpoint


In [9]:
model.train(False)
shared_layer_model.train(False)

ActorCriticSeparateWeights(
  (obs_normalizer): ObservationNormalizer()
  (returns_normalizer): RecursiveScriptModule(original_name=RunningMeanStdInPlace)
  (actor_encoder): ScaledNet(
    (encoders): ModuleDict()
    (crop): Crop()
    (extract_crop_representation): Sequential(
      (0): Conv2d(32, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ELU(alpha=1.0)
      (3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ELU(alpha=1.0)
      (6): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (8): ELU(alpha=1.0)
      (9): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (10): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=T

In [10]:
shared_layer_model(obs, shared_layer_rnn_states)

{'values': tensor(1.4808, grad_fn=<SqueezeBackward0>),
 'action_logits': tensor([[ -2.2515,  -4.4662,  -0.5797, -15.9201,  -5.3243,  -5.1846,  -4.3019,
          -13.0410, -18.8946, -14.5667, -23.9047,  -8.6105, -15.8444, -15.6841,
          -14.1679, -19.4248,  -9.3098,  -9.8468, -13.1831, -10.7035,   1.4208,
           -9.9007,  -9.9138,  -9.7891,  -0.1878,  -6.8914, -10.2171,  -8.3538,
          -13.7015,  -9.8290, -10.0067,  -9.7272,  -8.8228,  -8.7647,  -3.5368,
           -0.0560,  -7.4556,  -3.8298,   4.3536, -11.7893, -13.2708,  -9.8700,
           -9.7792, -14.2080, -11.0493, -18.7960, -10.0035,  -9.8139,  -3.3948,
            5.1919,  -9.7495,   0.0995,  -4.6482,  -9.8573, -10.1714,  -7.5221,
          -13.4136,  -8.8625, -14.6437,  -9.8632, -14.1097,  -2.9114,  -2.5040,
          -19.4561,  -9.7901,  -9.2849, -15.3096, -12.5556,  -9.7538,  -7.3811,
           -9.6222,  -9.8859, -10.6870, -17.8891, -15.7483,  -1.5771,  -9.6279,
           -9.6801,  -9.8373,  -9.8153,  -9.6897

In [11]:
model(obs, rnn_states)

{'values': tensor(0.5157, grad_fn=<SqueezeBackward0>),
 'action_logits': tensor([[ -2.2515,  -4.4662,  -0.5797, -15.9201,  -5.3243,  -5.1846,  -4.3019,
          -13.0410, -18.8946, -14.5667, -23.9047,  -8.6105, -15.8444, -15.6841,
          -14.1679, -19.4248,  -9.3098,  -9.8468, -13.1831, -10.7035,   1.4208,
           -9.9007,  -9.9138,  -9.7891,  -0.1878,  -6.8914, -10.2171,  -8.3538,
          -13.7015,  -9.8290, -10.0067,  -9.7272,  -8.8228,  -8.7647,  -3.5368,
           -0.0560,  -7.4556,  -3.8298,   4.3536, -11.7893, -13.2708,  -9.8700,
           -9.7792, -14.2080, -11.0493, -18.7960, -10.0035,  -9.8139,  -3.3948,
            5.1919,  -9.7495,   0.0995,  -4.6482,  -9.8573, -10.1714,  -7.5221,
          -13.4136,  -8.8625, -14.6437,  -9.8632, -14.1097,  -2.9114,  -2.5040,
          -19.4561,  -9.7901,  -9.2849, -15.3096, -12.5556,  -9.7538,  -7.3811,
           -9.6222,  -9.8859, -10.6870, -17.8891, -15.7483,  -1.5771,  -9.6279,
           -9.6801,  -9.8373,  -9.8153,  -9.6897

In [11]:
shared_layer_model(obs, shared_layer_rnn_states)

{'values': tensor(-1.2610, grad_fn=<SqueezeBackward0>),
 'action_logits': tensor([[ -2.2515,  -4.4662,  -0.5797, -15.9201,  -5.3243,  -5.1846,  -4.3019,
          -13.0410, -18.8946, -14.5667, -23.9047,  -8.6105, -15.8444, -15.6841,
          -14.1679, -19.4248,  -9.3098,  -9.8468, -13.1831, -10.7035,   1.4208,
           -9.9007,  -9.9138,  -9.7891,  -0.1878,  -6.8914, -10.2171,  -8.3538,
          -13.7015,  -9.8290, -10.0067,  -9.7272,  -8.8228,  -8.7647,  -3.5368,
           -0.0560,  -7.4556,  -3.8298,   4.3536, -11.7893, -13.2708,  -9.8700,
           -9.7792, -14.2080, -11.0493, -18.7960, -10.0035,  -9.8139,  -3.3948,
            5.1919,  -9.7495,   0.0995,  -4.6482,  -9.8573, -10.1714,  -7.5221,
          -13.4136,  -8.8625, -14.6437,  -9.8632, -14.1097,  -2.9114,  -2.5040,
          -19.4561,  -9.7901,  -9.2849, -15.3096, -12.5556,  -9.7538,  -7.3811,
           -9.6222,  -9.8859, -10.6870, -17.8891, -15.7483,  -1.5771,  -9.6279,
           -9.6801,  -9.8373,  -9.8153,  -9.689

In [12]:
rnn_states.shape

torch.Size([1, 3476])

In [13]:
model(obs, rnn_states)

{'values': tensor(0.1449, grad_fn=<SqueezeBackward0>),
 'action_logits': tensor([[ -2.2515,  -4.4662,  -0.5797, -15.9201,  -5.3243,  -5.1846,  -4.3019,
          -13.0410, -18.8946, -14.5667, -23.9047,  -8.6105, -15.8444, -15.6841,
          -14.1679, -19.4248,  -9.3098,  -9.8468, -13.1831, -10.7035,   1.4208,
           -9.9007,  -9.9138,  -9.7891,  -0.1878,  -6.8914, -10.2171,  -8.3538,
          -13.7015,  -9.8290, -10.0067,  -9.7272,  -8.8228,  -8.7647,  -3.5368,
           -0.0560,  -7.4556,  -3.8298,   4.3536, -11.7893, -13.2708,  -9.8700,
           -9.7792, -14.2080, -11.0493, -18.7960, -10.0035,  -9.8139,  -3.3948,
            5.1919,  -9.7495,   0.0995,  -4.6482,  -9.8573, -10.1714,  -7.5221,
          -13.4136,  -8.8625, -14.6437,  -9.8632, -14.1097,  -2.9114,  -2.5040,
          -19.4561,  -9.7901,  -9.2849, -15.3096, -12.5556,  -9.7538,  -7.3811,
           -9.6222,  -9.8859, -10.6870, -17.8891, -15.7483,  -1.5771,  -9.6279,
           -9.6801,  -9.8373,  -9.8153,  -9.6897